In [1]:
from configs import beta_vae
config = beta_vae.get_config()
config

batch_size: 1000000
dataset: Organoid
device: cuda
epochs: 10000
hidden_dims: !!python/tuple
- 32
- 32
- 32
in_features: 41
kld_weight: 0.0025
latent_dim: 2
learning_rate: 0.05
loss_type: beta
model: VAE
output_dir: ./logs/VanillaVAE/
save_loss_every_n_epochs: 25
seed: 12345
weight_decay: 0.0

In [2]:
from models import BetaVAE
config.hidden_dims = [32,16,8]
model = BetaVAE(config).to(config.device)

In [3]:
model

BetaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=41, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=16, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_mu): Linear(in_features=8, out_features=2, bias=True)
  (fc_var): Linear(in_features=8, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_r

In [5]:
from datasets import OrganoidDataset
data = OrganoidDataset()

X_train,y_train = data.train
X_val,y_val = data.val

In [6]:
import torch
print(f'Memory allocated:{torch.cuda.memory_allocated()}')
print(f'Max memory allocated:{torch.cuda.max_memory_allocated()}')

Memory allocated:201714688
Max memory allocated:201718272


In [9]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )
optimizer.zero_grad()
outputs = model.forward(X_train)
loss = model.loss_function(*outputs, config=config)['loss']
loss.backward()
optimizer.step()

In [15]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/vanilla_vae.py --config.epochs=10000')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 10000/10000 [14:01<00:00, 11.88it/s]


Finished Training


0

In [1]:
from configs import wae_mmd
config = wae_mmd.get_config()
config

batch_size: 4096
dataset: Organoid
device: cuda
epochs: 10000
hidden_dims:
- 32
- 32
- 32
in_features: 41
kernel_type: imq
latent_dim: 2
latent_var: 1
learning_rate: 0.005
model: WAE_MMD
reg_weight: 1
save_loss_every_n_epochs: 10
seed: 12345
weight_decay: 0.0

In [2]:
from models import WAE_MMD
config.hidden_dims = [32,16,8]
model = WAE_MMD(config).to(config.device)
model

WAE_MMD(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=41, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=16, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_z): Linear(in_features=8, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
 

In [3]:
from datasets import OrganoidDataset

data = OrganoidDataset()

X_train, y_train = data.train
X_val, y_val = data.val

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
import torch

X_train_batches = torch.split(X_train,split_size_or_sections=config.batch_size)
X_val_batches = torch.split(X_train,split_size_or_sections=config.batch_size)

In [5]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )
optimizer.zero_grad()
outputs = model.forward(X_train_batches[0])
loss = model.loss_function(*outputs, config=config)['loss']
loss.backward()
optimizer.step()

In [21]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/wae_mmd.py --config.epochs=200')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345


CUDA available:True


100%|██████████| 200/200 [17:26<00:00,  5.23s/it]


Finished Training


0

In [10]:
import os
for lr in [0.08,0.09,0.06]:
    os.system(f'python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
              f'--config=configs/beta_vae.py '
              f'--config.output_dir=logs/VanillaVAE/grid_example '
              f'--config.hidden_dims="(41,41,41,41)" '
              f'--config.learning_rate={lr} ')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

CUDA available:True


100%|██████████| 10000/10000 [25:46<00:00,  6.47it/s]


Finished Training


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

...

Finished Training


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

CUDA available:True


100%|██████████| 10000/10000 [25:16<00:00,  6.59it/s]


Finished Training


In [9]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/disentangled_beta_vae.py')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

CUDA available:True


100%|██████████| 10000/10000 [14:18<00:00, 11.64it/s]


Finished Training


0

In [8]:
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
          '--config=configs/beta_vae.py '
          '--config.kld_weight=0.25 '
          '--config.hidden_dims="(41,32,32,32)"')

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 1/10000 [00:00<32:22,  5.15it/s]

CUDA available:True


100%|██████████| 10000/10000 [19:31<00:00,  8.54it/s]


Finished Training


0

In [13]:
import os
dims = ["(64,64)","(64,64,64)","(64,64,64,64)","(64,64,64,64,64)"]
for dim in dims:
    os.system(f'python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
              f'--config=configs/beta_vae.py '
              f'--config.output_dir=logs/VanillaVAE/dims2 '
              f'--config.hidden_dims="{dim}" '
              f'--config.learning_rate=0.07 ')

Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

CUDA available:True


100%|██████████| 10000/10000 [22:47<00:00,  7.31it/s]


Finished Training


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 12345
  0%|          | 0/10000 [00:00<?, ?it/s]

...

Traceback (most recent call last):
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 144, in <module>
    app.run(main)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 31, in main
    data = OrganoidDataset()
  File "/home/egor/PycharmProjects/deep_dr/datasets.py", line 38, in __init__
    self.train = X[train_idx].clone().to(device), y[train_idx].clone().to(device)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/cuda/__init__.py", line 217, in _lazy_init
    torch._C._cuda_init()
RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the availa

CUDA available:False


Traceback (most recent call last):
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 144, in <module>
    app.run(main)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 31, in main
    data = OrganoidDataset()
  File "/home/egor/PycharmProjects/deep_dr/datasets.py", line 38, in __init__
    self.train = X[train_idx].clone().to(device), y[train_idx].clone().to(device)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/cuda/__init__.py", line 217, in _lazy_init
    torch._C._cuda_init()
RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the availa

CUDA available:False


Traceback (most recent call last):
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 144, in <module>
    app.run(main)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/home/egor/PycharmProjects/deep_dr/experiment.py", line 31, in main
    data = OrganoidDataset()
  File "/home/egor/PycharmProjects/deep_dr/datasets.py", line 38, in __init__
    self.train = X[train_idx].clone().to(device), y[train_idx].clone().to(device)
  File "/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/cuda/__init__.py", line 217, in _lazy_init
    torch._C._cuda_init()
RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the availa

In [8]:
import os
os.system(f'python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
          f'--config=configs/beta_vae.py '
          f'--config.output_dir=logs/profiling_test '
          f'--config.hidden_dims="(64,64,64,64,64)" '
          f'--config.learning_rate=0.03 '
          f'--config.epochs=1000 '
          f'--config.batch_size=4096')

Global seed set to 12345
  0%|          | 0/1000 [00:00<?, ?it/s]

CUDA available:True


100%|██████████| 1000/1000 [26:55<00:00,  1.62s/it]


Memory allocated:203261440
Max memory allocated:239602176
Finished Training in 0:26:55.455607


0

In [1]:
import os
for lr in [0.05,0.075,0.01,0.02,0.03]:
    os.system(f'python3 /home/egor/PycharmProjects/deep_dr/experiment.py '
          f'--config=configs/beta_vae.py '
          f'--config.output_dir=logs/BetaVAE/grid1 '
          f'--config.hidden_dims="(64,64,64,64,64)" '
          f'--config.learning_rate={lr} '
          f'--config.epochs=1000 '
          f'--config.batch_size=4096 '
              f'&')